In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO
import numpy as np
from pathlib import Path
colors = ["b", "g", "r", "y"]

In [ ]:
solved_vs_time_data = {}
for file in sorted(Path.cwd().glob("*Witness*")):
    name = str(file.stem)
    solved_vs_time_data[name] = pd.read_csv(file)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), constrained_layout=True)
plt.xscale("linear")
plt.xticks
plt.title("Problems vs. Rel. Wall Time")
plt.xlabel("Relative wall time (s)")
plt.ylabel("Unique problems\n solved", rotation=0, labelpad=50)
for i, (name, data) in enumerate(solved_vs_time_data.items()):
    name = name.split("_")
    label = name[1].split("-")[0] + " " + name[2]
    xs = data["Wall time"] - data["Wall time"].min()
    ax.plot(xs, data["Value"], label=label, color=colors[i])
ax.legend(loc="lower right")
plt.savefig("solved_vs_time.jpg", dpi=192, transparent=False)

In [ ]:
epoch_data = {}
for pth in Path("../wit_4w4c_tb/").glob("*"):
    run_name = str(pth.stem)
    epochs = [pd.read_csv(ep)  for ep in pth.glob("*epoch*.csv")]
    epoch_data[run_name] = epochs

In [ ]:
runs = {}
for d in sorted(Path("../wit_4w4c_tb/").iterdir()):
    run_name = str(d.stem)
    outputs = (d / "output.txt").open("r").readlines()
    outputs = [l.lstrip("|").replace("|\n","").replace(" ","") for l in outputs if l.startswith("| ")]
    runs[run_name] = outputs

In [ ]:
plot_data = {}
for run_name, data in runs.items():
    solved = set()
    num_expanded_this_batch = 0
    total_expansions = 0
    xs = []
    ys = []
    first_row = True
    # since we don't process the a batch until encountering the next batches header
    data.append('P')
    for row in data:
        if row[0] == "P": # new batch
            if not first_row: # at least one batch has been processed
                total_expansions += num_expanded_this_batch
                ys.append(len(solved))
                xs.append(total_expansions)
                num_expanded_this_batch = 0
            else:
                first_row = False
            continue
        
        # 'ProblemId|SolutionLength|NumExpanded|NumGenerated|Time'
        row_data = row.split("|")
        
        problem_id = int(row_data[0])
        problem_solved = int(row_data[1]) > 0
        if problem_solved and problem_id not in solved:
            solved.add(problem_id)
            
        num_expanded = int(row_data[2])
        num_expanded_this_batch += num_expanded
        
        idx_in_batch += 1
    plot_data[run_name] = {"xs": xs, "ys": ys}

In [ ]:
for scale in ["log", "linear"]:
    fig, ax = plt.subplots(figsize=(6,4), constrained_layout=True)
    plt.xscale(scale)
    ax.set_facecolor("white")
    ax.yaxis.set_figure
    plt.title(f"Problems Solved vs. ({scale}) Nodes Expanded")
    plt.xlabel("Nodes expanded")
    plt.ylabel("Unique problems\n solved", rotation=0, labelpad=50)
    for i, (name, data) in enumerate(plot_data.items()):
        name = name.split("_")
        label = name[1].split("-")[0] + " " + name[2]
        ax.plot(data["xs"], data["ys"], label=label, color=colors[i])
    plt.legend(loc="lower right")
    plt.savefig(f"solved_vs_{scale}_expanded.jpg", dpi=192, transparent=False)